In [43]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pickle


In [44]:
##### scale your data ######
##### MinMaxScaler: This estimator scales and translates each feature individually into (0,1) #######
##### StandardScaler: Standardize features by removing the mean and scaling to unit variance #####
##### RobustScaler: Scale features using statistics that are robust to outliers ######

#scaler = [MinMaxScaler(),StandardScaler(), RobustScaler()]
scaler = [StandardScaler()]

In [56]:
###### Input data and label here ############
###### load your data here, separate features and label #######
###### X contains all the features and Y is label
 
df=pd.read_csv("df_feat_small.csv").drop('Unnamed: 0',axis=1).dropna().copy()
df=df.drop('P/B_RATIO_Q_0',axis=1)
#X=np.random.rand(100,100)
#Y=np.random.randint(2, size=100)

X=df[df.columns[-24:]]
X=X[X.columns[1:]]
Y=df[df.columns[1]]
Y=Y.astype(int)
X.columns.size
#Y
#X.replace([np.inf, -np.inf], np.nan)
#X.replace('0', 0.0001)
#Y.replace('0', 0.00001)
#X.dropna()

23

In [46]:
from sklearn.utils import resample

#Get data loaded
df_small = pd.read_csv('df_feat_small.csv')

#Create ETF lists 
list_small_etf = []
df_small_c = df_small[['DWAS', 'EWSC',
       'FNDA', 'IUSS', 'OMFS', 'PBSM', 'PSCD', 'PSCF', 'PSCH', 'PSCI', 'PSCT',
       'PSCU', 'RWJ', 'RZG', 'RZV', 'SLY', 'SLYG', 'SLYV', 'SMLV', 'SPMD',
       'SPSM', 'VB', 'VBK', 'VBR', 'VIOG', 'VIOO', 'VIOV', 'VSS', 'XSHD',
       'XSHQ', 'XSMO', 'XSVM']].copy()

#Create list of ETFs
for i in df_small_c.columns:
    list_small_etf.append(i)


#Function to Balance Scale Dataset
def upSample():
    
    for etf in list_small_etf:
        indexETF = list_small_etf.index(etf)
        list_small_etf[indexETF]
        
        #Count the number of ones and zeros
        numbFalse = (df_small[list_small_etf[indexETF]] == 0).sum()
        numbTrue = (df_small[list_small_etf[indexETF]] == 1).sum()
        
        #If logic to use the largest value between number of Trues and False
        if numbFalse > numbTrue:
            upSample = numbFalse
        else:
            upSample = numbTrue

        # Separate majority and minority classes
        df_majority = df_small[df_small[list_small_etf[indexETF]] == 0]
        df_minority = df_small[df_small[list_small_etf[indexETF]] == 1]

        # Upsample minority class
        df_minority_upsampled = resample(df_minority, 
                                             replace=True,     # sample with replacement
                                             n_samples = upSample,    # to match majority class
                                             random_state=123) # reproducible results

            #Combine majority class with upsampled minority class
        df_upsampled = pd.concat([df_majority, df_minority_upsampled])

            # Display new class counts
            #print(df_upsampled.etf.value_counts())
        
        # Separate input features (X) and target variable (y)
        y = df_upsampled[list_small_etf[indexETF]]
        X = df_upsampled.drop(list_small_etf[indexETF], axis=1)

        #Split data into Training and Test 
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1)
        re=X_train, X_test, y_train, y_test
        return re

In [52]:
#X.isnull().sum()
#X.loc[X['CURRENT_RATIO_Q_0'] == '']
X_train, X_test, y_train, y_test=upSample()
X_train

,Unnamed: 0,TICKER,EWSC,FNDA,IUSS,OMFS,PBSM,PSCD,PSCF,PSCH,...,RYU,VAW,VDC,VNQ,XLC,XLE,XLI,XLK,XLV,XLY
109,109,PPBI,True,False,False,False,True,False,True,False,...,-0.316272,0.006782,-0.287225,-0.585512,-0.060732,0.159896,0.507884,-0.730749,-0.029519,0.448128
748,748,TDOC,False,False,True,False,True,False,False,False,...,-0.186972,-0.416549,-0.809563,-0.003518,0.511500,0.259348,0.250754,-0.042184,1.486591,0.268461
498,498,CACI,False,False,True,False,True,False,False,False,...,0.047576,-0.064037,0.005780,0.065793,0.105350,-0.177574,0.288551,0.066781,0.147575,0.020550
187,187,NCBS,False,False,False,True,True,False,False,False,...,-0.018645,-0.005029,-0.188209,-0.133103,0.189274,0.114882,0.041809,-0.350571,0.063603,0.299715
219,219,CVCO,True,True,True,False,True,True,False,False,...,0.181375,0.032087,-0.148152,-0.144205,-0.066573,-0.060808,0.326536,-0.395451,-0.136530,0.549724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,228,ICFI,False,True,True,False,True,False,False,False,...,0.247832,-0.106333,-0.012864,-0.351644,-0.187187,-0.036497,0.214385,-0.044131,0.187966,0.297935
109,109,PPBI,True,False,False,False,True,False,True,False,...,-0.316272,0.006782,-0.287225,-0.585512,-0.060732,0.159896,0.507884,-0.730749,-0.029519,0.448128
1114,1114,WABC,True,True,False,False,True,False,True,False,...,-0.119303,-0.099163,-0.023164,-0.413880,-0.088117,0.119329,0.458457,-0.516206,0.061388,0.235824
159,159,RDVT,False,False,False,False,False,False,False,False,...,0.525795,0.120491,-1.589662,0.077189,-0.252594,-0.047858,0.527061,-0.072488,0.414791,-0.095357


In [11]:
###### use Random Forest as classifier #####
clf=RandomForestClassifier(random_state=0)

In [12]:
###### cross validation: separate dataset ##########
kfold= StratifiedKFold(n_splits=3, shuffle = True, random_state=0)


In [13]:
####set up pipline here ######
#### try different scaler here try [MinMaxScaler(),StandardScaler(), RobustScaler()] ####
steps = [('scaler', StandardScaler()),('rf',clf)]
pipeline= Pipeline(steps)

In [14]:
######## grid search for hyperparameter #########
######## Need to tune parameter, max_depth, min_samples_leaf and max_features #####
parameters = {
    #'rf__scaler': scaler,
    'rf__max_depth':  [3,4],#np.linspace(2,4,2),
    'rf__min_samples_leaf': [3,4],#np.linspace(1,5,5),
    'rf__max_features': [5,6,7],#np.linspace(2,4,2),
    #"rf__criterion": ["gini", "entropy"]
    
    
    
}
parameters


{'rf__max_depth': [3, 4],
 'rf__min_samples_leaf': [3, 4],
 'rf__max_features': [5, 6, 7]}

In [15]:
##### model ####
mdl=GridSearchCV(pipeline,param_grid = parameters,n_jobs=-1,cv=kfold,scoring='roc_auc',iid=False)

In [16]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'scaler', 'rf', 'scaler__copy', 'scaler__with_mean', 'scaler__with_std', 'rf__bootstrap', 'rf__ccp_alpha', 'rf__class_weight', 'rf__criterion', 'rf__max_depth', 'rf__max_features', 'rf__max_leaf_nodes', 'rf__max_samples', 'rf__min_impurity_decrease', 'rf__min_impurity_split', 'rf__min_samples_leaf', 'rf__min_samples_split', 'rf__min_weight_fraction_leaf', 'rf__n_estimators', 'rf__n_jobs', 'rf__oob_score', 'rf__random_state', 'rf__verbose', 'rf__warm_start'])

In [17]:
##### fit model with data #####
mdl.fit(X,Y)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=0, shuffle=True),
             error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('scaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('rf',
                                        RandomForestClassifier(bootstrap=True,
                                                               ccp_alpha=0.0,
                                                               class_weight=None,
                                                               criterion='gini',
                                                               max_depth=None,
                                                               max_features='auto',
                                                              

In [18]:
##### predict ######
result=mdl.predict(X)

In [36]:
####### Load test set here ######



In [19]:
##### Here we generate a matrix , 32 columns gives predicted result of 32 classifier ######
Predict_M=df[df.columns[0]].T
for i in range(32):
    
    Y=df[df.columns[i+1]]
    mdl.fit(X,Y)
    result=mdl.predict(X)
    Predict_M=np.vstack((Predict_M, result.T))
    filename = 'model'+str(i+1)+'sav'
    pickle.dump(model, open(filename, 'wb'))
    
    

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning
/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


In [63]:
Predict_M.T

array([['IBKR', False, False],
       ['AGR', False, False],
       ['SSNC', False, False],
       ...,
       ['KEM', False, False],
       ['MCB', False, False],
       ['AFH', False, False]], dtype=object)